In [11]:
7

7

## Gdrive

In [2]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 12.6/12.6 MB 1.2 MB/s eta 0:00:00
     ------------------------------------ 138.3/138.3 kB 547.2 kB/s eta 0:00:00
     -------------------------------------- 96.9/96.9 kB 692.9 kB/s eta 0:00:00
     ------------------------------------ 189.2/189.2 kB 602.5 kB/s eta 0:00:00
     ------------------------------------ 229.1/229.1 kB 500.9 kB/s eta 0:00:00
     -------------------------------------- 48.8/48.8 kB 189.8 kB/s eta 0:00:00
     ------------------------------------ 413.4/413.4 kB 806.2 kB/s eta 0:00:00
     ------------------------------------ 151.7/151.7 kB 822.4 kB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [23]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import os
from googleapiclient.http import MediaIoBaseDownload
import io
import pandas as pd

# Scopes required by the app

def get_service():
    SCOPES = ['https://www.googleapis.com/auth/drive.file']

    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('drive', 'v3', credentials=creds)
    return service


def gdrive_file_upload(filename, gdrive_folder_id):
    service = get_service()

    # File to upload
    file_metadata = {'name': filename, 'parents': [gdrive_folder_id]}
    media = MediaFileUpload(filename, mimetype='application/zip')
    file = service.files().create(body=file_metadata,
                                    media_body=media,
                                    fields='id').execute()
    file_id = file.get('id')
    csv_file = 'log.csv'

    df = pd.DataFrame(columns=['File Name', 'File ID', 'time'])
    df = df.append({'File Name': filename, 'File ID': file_id, 'time': pd.Timestamp.now()}, ignore_index=True)
    if os.path.exists(csv_file):
        df.to_csv(csv_file, mode='a', header=False, index=False)
    
    return file.get('id')

def gdrive_file_download(filename, gdrive_file_id):
    service = get_service()
    request = service.files().get_media(fileId=gdrive_file_id)
    
    # Replace 'filename' with the path where you want to save the downloaded file
    fh = io.FileIO(filename, mode='wb')
    downloader = MediaIoBaseDownload(fh, request)
    
    done = False
    while not done:
        status, done = downloader.next_chunk()
        print("Download Progress: {0}".format(status.progress() * 100))

    fh.close()
    print("File has been downloaded successfully.")



In [26]:
file = 'data/example1.zip'
gdrive_folder_id = '1te7ArAHuMquqmWhjmO-FniTE47O4mts7'
uploaded_file_id = gdrive_file_upload(file, gdrive_folder_id)
print(f"File has been uploaded successfully. File ID: {uploaded_file_id}")

File has been uploaded successfully. File ID: 1UIrOrxgAfFdjPdUugqhTBojY29CtXGEm


C:\Users\user\AppData\Local\Temp\ipykernel_12816\2523878073.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'File Name': filename, 'File ID': file_id, 'time': pd.Timestamp.now()}, ignore_index=True)


In [27]:
file = 'data/filename.zip'
gdrive_file_id = uploaded_file_id
gdrive_file_download(file, gdrive_file_id)

Download Progress: 100.0
File has been downloaded successfully.


## Azure

In [28]:
%pip install msal

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

     ------------------------------------ 102.2/102.2 kB 309.5 kB/s eta 0:00:00


In [1]:
from msal import PublicClientApplication
import requests

def getEndpoint(access_token, domain, sites, siteName, path):

    # Assuming you have an access token
    headers = {
        "Authorization": access_token
    }

    # Get site by URL
    URL = f"https://graph.microsoft.com/v1.0/sites/{domain}.sharepoint.com:/{sites}/{siteName}"
    site_response = requests.get(URL, headers=headers)
    site_id = site_response.json().get('id')

    # Get default drive of the site
    drive_response = requests.get(
        f'https://graph.microsoft.com/v1.0/sites/{site_id}/drive',
        headers=headers
    )
    drive_id = drive_response.json().get('id')

    # print("Site ID:", site_id)
    # print("Drive ID:", drive_id)
    endpoint = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drives/{drive_id}/root:/{path}:/content"

    return endpoint


def onedrive_file_upload(token_response, domain, sites, siteName, onedrive_path, localFile):
    if "access_token" in token_response:
        access_token = token_response['access_token']

        endpoint = getEndpoint(access_token, domain, sites, siteName, onedrive_path)

        headers = {
            "Authorization": "Bearer " + access_token,
            "Content-Type": "application/zip"  # Change according to your file type
        }

        # Read the file content to upload
        file_content = open(localFile, 'rb').read()  # Modify the file path and mode as needed

        # Make the HTTP request to upload the file
        response = requests.put(endpoint, headers=headers, data=file_content)

        if response.status_code == 201:
            print("File uploaded successfully.")
            print(response.json())
        elif response.status_code == 200:
            print("Failed to upload file. File already exists.")
        else:
            print("Failed to upload file.")
            print(response.status_code, response.text)
    else:
        print("Failed to obtain token.")
        print(token_response.get("error"), token_response.get("error_description"))


def onedrive_file_download(token_response, onedrive_path, localFile):
    if "access_token" in token_response:
        access_token = token_response['access_token']

        # Build the endpoint for downloading the file from OneDrive
        endpoint = f"https://graph.microsoft.com/v1.0/me/drive/root:/{onedrive_path}:/content"

        headers = {
            "Authorization": "Bearer " + access_token,
        }

        # Make the HTTP request to download the file
        response = requests.get(endpoint, headers=headers, stream=True)

        if response.status_code == 200:
            # Save the downloaded content to a file
            with open(localFile, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print("File downloaded successfully.")
        else:
            print("Failed to download file.")
            print(response.status_code, response.text)
    else:
        print("Failed to obtain token.")
        print(token_response.get("error"), token_response.get("error_description"))

In [3]:
from msal import PublicClientApplication

domain = "iiitaphyd-my"
sites = "personal"
siteName = "rudra_dhar_research_iiit_ac_in"
onedrive_path = "IIIT/sem4/SE/project_3/data"

app = PublicClientApplication(
    client_id='cae82036-061d-4c94-a773-f9694b8042f0',  # Replace with your client ID
    authority='https://login.microsoftonline.com/031a3bbc-cf7c-4e2b-96ec-867555540a1c',  # Replace with your tenant ID
)

# Include the required scopes
scopes = ["Files.ReadWrite", "User.Read"]
token_response = app.acquire_token_interactive(scopes)

In [4]:
localFileToUpload = 'data/example1.zip'
onedrive_file_upload(token_response, domain, sites, siteName, onedrive_path, localFileToUpload)

Failed to upload file.
400 {"error":{"code":"invalidRequest","message":"A valid name must be provided.","innerError":{"date":"2024-04-21T11:02:23","request-id":"3a72bd82-e029-4d8d-bb8f-f29aeee0b648","client-request-id":"3a72bd82-e029-4d8d-bb8f-f29aeee0b648"}}}


In [ ]:
localFileToDownload = 'data/downloaded_example7.zip'
onedrive_file_download(token_response, onedrive_path, localFileToDownload)